# Troubleshooting: Colab, .env, and GitHub 404s

- **Colab ignores local `.env` files.** Use the prompt at the top of the notebook to set your `GOOGLE_CLOUD_PROJECT`, or run:

  ```python
  import os
  os.environ['GOOGLE_CLOUD_PROJECT'] = 'your-project-id'
  ```

- **If you see a `404 Not Found` when opening via the "Open in Colab" badge:** the badge may point to a branch or path that doesn't contain the notebook. Fix options:
  - Push the notebooks to the branch referenced by the badge (commonly `main`).
  - Or update the badge URL to the repository branch that has the notebooks.

- **If Colab prompts for GitHub access or fails to load:** sign into Colab and authorize GitHub access, then retry using the badge or use `File → Open notebook → GitHub` and search the repo.

- **Workarounds:** if issues persist, use `File → Save a copy in Drive` from the Colab error view or open the notebook directly from GitHub and copy into Colab.

- **Transient errors:** GitHub API rate limits or transient failures can cause temporary 404s — wait a minute and retry if everything else looks correct.

## Prerequisites

- A Google account.
- Access to a Google Cloud Project with billing enabled (instructor-provided or student project).
- The Vertex AI API enabled for the project.
- (Recommended) The instructor may provide a project ID or a service account for class use.

## Quick Start (one-time in Colab)

1. Click the **Open In Colab** badge at the top of any notebook in the repository.
2. Run the first setup cell to install dependencies and authenticate.
3. When prompted, paste your Google Cloud Project ID (or leave blank to use ADC).
4. After setup completes you can run the lesson cells as shown in class.

In [ ]:
# Colab setup cell - Run this first in Colab
import sys
IN_COLAB = "google.colab" in sys.modules
if not IN_COLAB:
    print("Not running in Colab. This cell is intended for Google Colab users.")
else:
    print("Running Colab setup...")
    # Install required packages quietly
    !pip install -q google-genai google-auth python-dotenv numpy
    from google.colab import auth
    auth.authenticate_user()
    try:
        PROJECT_ID = input("Enter your Google Cloud Project ID (press Enter to use default ADC): ").strip()
    except Exception:
        PROJECT_ID = ""
    if PROJECT_ID:
        import os
        os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
    print("Authentication complete. If you provided a Project ID, it was set as GOOGLE_CLOUD_PROJECT.")

In [ ]:
# Verify GenAI client initialization
import os
import google.auth
from google import genai
from google.genai import types

creds, project = google.auth.default()
project = os.environ.get("GOOGLE_CLOUD_PROJECT", project)

try:
    client = genai.Client(vertexai=True, project=project, location="us-central1")
    print(f"GenAI client initialized. Using project: {project}")
except Exception as e:
    print("Failed to initialize GenAI client:", e)
    print("Common fixes: ensure Vertex AI API is enabled and your account has required IAM roles.")

## Enabling APIs & Permissions

If you haven't enabled Vertex AI for your project, run this (locally or in Cloud Shell):

```bash
# Enable Vertex AI (run from Cloud Shell or local environment with gcloud authenticated)
gcloud services enable aiplatform.googleapis.com --project=YOUR_PROJECT_ID
```

Recommended roles for students: project `Editor` or a narrower custom role that includes Vertex AI and storage permissions. If using service accounts, grant the account appropriate Vertex AI permissions.

## Billing and Quotas

- Colab runs compute on Google infrastructure; if you call Vertex AI (Gemini) you'll incur charges on the linked Google Cloud Project.
- Confirm billing is enabled for the Project ID you use.
- Monitor usage in the Google Cloud Console billing section.

## Troubleshooting

- "Permission denied" or "403": make sure your account has access to the chosen project and Vertex AI is enabled.
- "Client initialization" errors: ensure `GOOGLE_CLOUD_PROJECT` is set and the project exists.
- If you see networking or DNS issues when installing packages in Colab, try restarting the runtime and re-running the setup cell.

If problems persist, capture the error message and share it with the instructor.

## Instructor Tips

- Consider providing a shared project or pre-configured service account to avoid student setup friction.
- Share a short list of required IAM roles and a link to your project's Vertex AI overview in the Cloud Console.
- If students will not have billing enabled, provide sample outputs or a demo notebook that uses local mocks instead of live API calls.